In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

In [18]:
df = pd.read_csv('data_2genre.csv')
df['label'] = (df['label'] == 2).astype(int)

In [19]:
df.head()

,filename,tempo,beats,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,pop.00024.au,117.453835,57,0.410990,0.207584,3301.781785,3175.656592,7302.597509,0.144527,-40.802069,...,-0.568772,0.552448,-1.981371,1.915755,0.076505,-1.270496,1.233082,1.493700,0.949931,0
1,pop.00099.au,78.302557,38,0.360462,0.138195,2765.747194,3222.713605,6873.391771,0.096015,-119.320605,...,1.566938,-2.932391,-1.609138,-0.200545,-2.429376,1.207988,5.848729,1.609469,7.580263,0
2,pop.00009.au,107.666016,50,0.454810,0.159711,3279.877315,3026.391118,6953.099622,0.172819,-35.277026,...,-0.518891,0.212918,-1.294660,-0.105626,-1.245879,-2.528455,0.571032,1.569434,-1.724873,0
3,pop.00078.au,103.359375,52,0.454698,0.223419,3124.176350,2970.320532,6618.134981,0.142721,-42.757179,...,3.020176,-1.242321,-3.524069,-0.325165,-1.443294,-1.864617,-0.031307,-0.729631,-0.987611,0
4,pop.00037.au,103.359375,51,0.451668,0.240985,3236.413205,2953.596262,6698.776160,0.164825,4.821123,...,1.753352,3.336132,2.352307,1.428845,-0.604926,-2.185691,-1.602394,-0.612146,-4.050126,0


In [31]:
X = df.iloc[:, 1:-1]
X_norm = normalize(X)
y = df.iloc[:, -1]

# Uncomment for no scaling
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
# X_train, X_test, y_train, y_test = X_train.T.values, X_test.T.values, y_train.T.values.reshape(1, 150), y_test.T.values.reshape(1, 50)

# Uncomment for scaled values
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, random_state = 0)
X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.values.reshape(1, 150), y_test.values.reshape(1, 50)

In [32]:
y_train

array([[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
        1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0,
        1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1]])

In [33]:
print ("X_train shape: " + str(X_train.shape))
print ("y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("y_test shape: " + str(y_test.shape))
print ("sanity check after reshaping: " + str(X_train[0:5,0]))

X_train shape: (28, 150)
y_train shape: (1, 150)
X_test shape: (28, 50)
y_test shape: (1, 50)
sanity check after reshaping: [9.57399190e-03 4.62029416e-03 4.18679444e-05 1.99767152e-05
 3.67688554e-01]


In [34]:
# X_mod = X_train.T.values

## Helper functions

Sigmoid

In [35]:
def sigmoid(z):
    s = 1 / (1 + np.exp(-(z)))    
    return s

Initializing weights and bias

In [36]:
def initialize_with_zeros(dim):
    w = np.zeros((dim, 1)) # n x 1
#     w = np.random.randn(dim, 1) * 0.1
    b = 0                  
    return w, b

In [37]:
def propagate(w, b, X, Y):

    m = X.shape[1]
#     print(f'm is {m}')
    A = sigmoid(np.dot(w.T, X) + b)                                    # compute activation
#     print(f'A is {A} and 1-A is {1-A}')
    cost = np.sum((Y * np.log(A)) + ((1-Y) * np.log(1-A))) * (-1/m)    # compute cost

    dz = (A - Y)                # derivative of J wrt to z
    dw = np.dot(X, dz.T) / m    # derivative of J wrt to w
    db = np.sum(dz) / m         # derivative of J wrt to b
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

In [38]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        
        
        # Cost and gradient calculation
        grads, cost = propagate(w, b, X, Y)
#         print(cost)
        # Retrieve derivatives from grads
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - (learning_rate * dw)
        b = b - (learning_rate * db)
        
        # Record the costs for every 100 iteration 
        if i % 100 == 0:
            costs.append(cost)
        
        # Print the cost every 100 training iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        
        
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs

In [39]:
def predict(w, b, X):
    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
#     print(w.shape)
    w = w.reshape(X.shape[0], 1)
    
    A = sigmoid(np.dot(w.T, X) + b)
    
    # Convert probabilities A[0,i] to actual predictions p[0,i]
#     print(A)
    Y_prediction = (A > 0.5).astype(float)
        
    return Y_prediction

In [40]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    
    # initialize parameters with zeros (≈ 1 line of code)
    w, b = initialize_with_zeros(X_train.shape[0])

    # Gradient descent 
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations=num_iterations, learning_rate=learning_rate)
    
    # Retrieve parameters w and b from dictionary "parameters"
    w = parameters["w"]
    b = parameters["b"]
    
    # Predict test/train set examples (≈ 2 lines of code)
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)


    # Print train/test Errors
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [41]:
model(X_train, y_train, X_test, y_test, num_iterations = 4000, learning_rate = 0.05, print_cost = True)

train accuracy: 92.66666666666667 %
test accuracy: 98.0 %


{'costs': [0.6931471805599454,
  0.6838032783237651,
  0.6751488156721739,
  0.6667787531159709,
  0.6586802997252429,
  0.6508432776329579,
  0.6432576190203011,
  0.6359134051483257,
  0.6288009140107651,
  0.6219106599703689,
  0.6152334256053502,
  0.6087602862996856,
  0.6024826282385453,
  0.5963921605405041,
  0.590480922282999,
  0.5847412851670193,
  0.579165952530869,
  0.5737479553694514,
  0.5684806459520078,
  0.5633576895631093,
  0.5583730548231961,
  0.5535210029789793,
  0.5487960764925561,
  0.5441930872022314,
  0.5397071042783425,
  0.5353334421538943,
  0.5310676485723596,
  0.5269054928631215,
  0.5228429545282792,
  0.5188762122022746,
  0.51500163302753,
  0.5112157624744208,
  0.50751531462196,
  0.5038971629060904,
  0.5003583313350594,
  0.4968959861656193,
  0.49350742802949454,
  0.49019008449636436,
  0.48694150305737505,
  0.483759344511673],
 'Y_prediction_test': array([[0., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0.,
         0., 1., 0.,